## Step 3 - Clustering Based on Price Polices 


### Installation and Setup

In [ ]:
%%capture
%pip install pandas plotly scipy

In [ ]:
import pandas as pd
import plotly.figure_factory as ff
from scipy.cluster.hierarchy import linkage
from plotly.subplots import make_subplots
import plotly.graph_objects as go

### 3.6 Identify the 150 hotels with the most data in the dataset and extract their records.


In [1]:
import pandas as pd

file_path = "../data/hotels_data_changed.csv"  
df = pd.read_csv(file_path)

hotel_counts = df['Hotel Name'].value_counts()
top_150_hotels = hotel_counts.head(150).index
filtered_df = df[df['Hotel Name'].isin(top_150_hotels)]

display(filtered_df)

,Snapshot ID,Snapshot Date,Checkin Date,Days,Original Price,Discount Price,Discount Code,Available Rooms,Hotel Name,Hotel Stars,DayDiff,WeekDay,DiscountDiff,DiscountPerc
2,1,2015-07-17,2015-08-13,5,4370,4240,1,3,The Peninsula New York,5,27,Thursday,130,2.974828
3,1,2015-07-17,2015-07-26,5,1739,1667,1,18,Eventi Hotel a Kimpton Hotel,4,9,Sunday,72,4.140311
4,1,2015-07-17,2015-08-12,5,1739,1672,1,3,Eventi Hotel a Kimpton Hotel,4,26,Wednesday,67,3.852789
5,1,2015-07-17,2015-08-07,5,1436,1345,1,9,Grand Hyatt New York,4,21,Friday,91,6.337047
6,1,2015-07-17,2015-08-09,5,1616,1525,1,5,Grand Hyatt New York,4,23,Sunday,91,5.631188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187842,99,2015-08-15,2015-09-03,5,1895,1735,4,-1,Cassa Hotel 45th Street,4,19,Thursday,160,8.443272
187843,99,2015-08-15,2015-09-04,5,1995,1935,4,-1,Cassa Hotel 45th Street,4,20,Friday,60,3.007519
187845,99,2015-08-15,2015-08-20,5,1456,1397,4,8,Hyatt Union Square New York,4,5,Thursday,59,4.052198
187846,99,2015-08-15,2015-09-02,5,1705,1648,4,2,Hyatt Union Square New York,4,18,Wednesday,57,3.343109


### 3.7 Find the 40 most common check-in dates  in the dataset and extract their records.


In [2]:
checkin_counts = filtered_df['Checkin Date'].value_counts()
top_40_checkin_dates = checkin_counts.head(40).index
filtered_checkin_df = filtered_df[filtered_df['Checkin Date'].isin(top_40_checkin_dates)]

display(filtered_checkin_df)

,Snapshot ID,Snapshot Date,Checkin Date,Days,Original Price,Discount Price,Discount Code,Available Rooms,Hotel Name,Hotel Stars,DayDiff,WeekDay,DiscountDiff,DiscountPerc
2,1,2015-07-17,2015-08-13,5,4370,4240,1,3,The Peninsula New York,5,27,Thursday,130,2.974828
4,1,2015-07-17,2015-08-12,5,1739,1672,1,3,Eventi Hotel a Kimpton Hotel,4,26,Wednesday,67,3.852789
7,1,2015-07-17,2015-08-12,5,1256,1183,1,5,Grand Hyatt New York,4,26,Wednesday,73,5.812102
8,1,2015-07-17,2015-08-13,5,1256,1201,1,54,Grand Hyatt New York,4,27,Thursday,55,4.378981
29,1,2015-07-17,2015-08-12,5,3763,3378,1,39,Park Hyatt New York,5,26,Wednesday,385,10.231199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187834,99,2015-08-15,2015-08-19,5,1095,1025,4,-1,DoubleTree by Hilton New York City - Financial...,4,4,Wednesday,70,6.392694
187836,99,2015-08-15,2015-08-27,5,995,865,4,-1,DoubleTree by Hilton New York City - Financial...,4,12,Thursday,130,13.065327
187839,99,2015-08-15,2015-08-19,5,1445,1345,4,-1,Cassa Hotel 45th Street,4,4,Wednesday,100,6.920415
187840,99,2015-08-15,2015-08-26,5,1545,1435,4,-1,Cassa Hotel 45th Street,4,11,Wednesday,110,7.119741


### 3.8 160-dimensional feature vector

**Task**

Build a 160-dimensional feature vector for each hotel based on its discount pricing behavior. Each vector is constructed by:
- Filtering the top 150 hotels (by record count) and the top 40 checkin dates.
- For each hotel, extracting 4 discount prices (one per discount code) for each of the 40 checkin dates. If several snapshots exist, we will select the one's with minimal prices.
- If no data is available for a specific (checkin date, discount code) combination, mark it with `-1`.

**Plan**

1. **Group the Data:**  
   Group the filtered data by **Hotel Name**, **Checkin Date**, and **Discount Code**. For each group, compute the minimum discount price, ensuring that only the best (lowest) price per combination is selected.

2. **Pivot to Wide Format:**  
   Transform the grouped data into a wide format where:
   - Each row represents a single hotel.
   - Each column represents a unique (Checkin Date, Discount Code) combination, totaling 160 columns (40 dates × 4 codes).

3. **Fill Missing Data:**  
   - Reindex the pivoted DataFrame so that every hotel has all 160 combinations, filling missing entries with `-1`.

In [3]:
# 1. Group by Hotel Name, Checkin Date, and Discount Code and select the minimum Discount Price.
grouped = (
    filtered_checkin_df
    .groupby(['Hotel Name', 'Checkin Date', 'Discount Code'])['Discount Price']
    .min()
    .reset_index()
)

# 2. Pivot the DataFrame so that:
#    - The index is 'Hotel Name'
#    - The columns are a MultiIndex with levels (Checkin Date, Discount Code)
#    - The values are the minimum discount prices.
pivot_df = grouped.pivot_table(index='Hotel Name',
                               columns=['Checkin Date', 'Discount Code'],
                               values='Discount Price')


# 3. Reindex the columns so that all 40 checkin dates and 4 discount codes are present.
#    Use the top_40_checkin_dates (from your earlier filtering) and the list [1, 2, 3, 4] for discount codes. 
all_combinations = pd.MultiIndex.from_product([top_40_checkin_dates, [1, 2, 3, 4]],
                                                names=['Checkin Date', 'Discount Code'])

pivot_df = pivot_df.reindex(columns=all_combinations, fill_value=-1)
pivot_df = pivot_df.fillna(-1)


pivot_df.columns = [
    col if isinstance(col, str) else f"{col[0]} - {col[1]}"
    for col in pivot_df.columns
]
pivot_df = pivot_df.reset_index()

print(pivot_df.shape[0]) # Note we have 149 hotels instead of 150 - solution in next cell
display(pivot_df)

149


,Hotel Name,2015-11-11 - 1,2015-11-11 - 2,2015-11-11 - 3,2015-11-11 - 4,2015-10-14 - 1,2015-10-14 - 2,2015-10-14 - 3,2015-10-14 - 4,2015-11-04 - 1,...,2015-11-03 - 3,2015-11-03 - 4,2015-08-13 - 1,2015-08-13 - 2,2015-08-13 - 3,2015-08-13 - 4,2015-11-25 - 1,2015-11-25 - 2,2015-11-25 - 3,2015-11-25 - 4
0,Aloft Harlem,-1.0,-1.0,-1.0,-1.0,1381.0,-1.0,1164.0,-1.0,1317.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,Andaz 5th Avenue - a concept by Hyatt,-1.0,1982.0,2375.0,-1.0,-1.0,2435.0,-1.0,-1.0,2051.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,1339.0,-1.0,1502.0,2103.0,2103.0
2,Andaz Wall Street - a concept by Hyatt,1078.0,1305.0,1430.0,-1.0,1625.0,1925.0,1900.0,-1.0,-1.0,...,2060.0,2185.0,-1.0,-1.0,-1.0,-1.0,1013.0,1029.0,990.0,-1.0
3,Baccarat Hotel and Residences New York,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,Bentley Hotel,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2023.0,2041.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,Wyndham Garden Brooklyn Sunset Park,-1.0,803.0,837.0,-1.0,-1.0,1042.0,1089.0,1170.0,-1.0,...,786.0,-1.0,1135.0,1135.0,-1.0,-1.0,-1.0,905.0,922.0,-1.0
145,Wyndham Garden Chinatown,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
146,Wyndham Garden Long Island City Manhattan View,734.0,752.0,-1.0,-1.0,-1.0,-1.0,1060.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,769.0,-1.0,-1.0
147,Wyndham New Yorker,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


#### Identifying Missing Hotel

**Verifying Missing Hotel Data**

After filtering and pivoting the data, we expect to have 150 hotels, but only 149 appear in our pivot table. This indicates that one (or more) of the top 150 hotels has no records for the top 40 check-in dates used in our analysis.

The code above does the following:
1. **Identify Missing Hotels:**  
   It compares the complete list of top 150 hotels (`top_150_hotels`) with the hotel names present in the pivoted DataFrame (`pivot_df`). Any hotel that is not present is added to the `missing_hotels` list.

2. **Check Data for Each Missing Hotel:**  
   For each missing hotel, it filters `filtered_checkin_df` (which already contains only records from the top 40 check-in dates) to see if there are any records for that hotel.  
   - If the resulting DataFrame is empty, it confirms that the hotel indeed has no data for those check-in dates.  
   - This explains why the hotel did not appear in the pivot table.

By verifying that the missing hotel has no records in the filtered data, we can conclude that the drop in the number of hotels is due to the absence of data for those check-in dates rather than an error in our processing pipeline.



In [4]:
# Assuming you have already defined:
# - top_150_hotels: the complete list of top 150 hotel names.
# - pivot_df: the pivoted DataFrame after grouping and filtering.
# - filtered_checkin_df: the DataFrame filtered by top 40 check-in dates.
#
# And the missing hotels are identified as:
missing_hotels = [hotel for hotel in top_150_hotels if hotel not in pivot_df['Hotel Name'].values]
print("Missing hotels:", missing_hotels)

# For each missing hotel, check if there is any record in the filtered_checkin_df.
for hotel in missing_hotels:
    hotel_records = filtered_checkin_df[filtered_checkin_df['Hotel Name'] == hotel]
    print(f"\nRecords for missing hotel '{hotel}':")
    print(hotel_records)  # This should print an empty DataFrame if no data is present.


Missing hotels: ['Embassy Suites Newark Airport']

Records for missing hotel 'Embassy Suites Newark Airport':
Empty DataFrame
Columns: [Snapshot ID, Snapshot Date, Checkin Date, Days, Original Price, Discount Price, Discount Code, Available Rooms, Hotel Name, Hotel Stars, DayDiff, WeekDay, DiscountDiff, DiscountPerc]
Index: []


### 3.9 Normalize 0-100


**Task**

For each hotel, we have a 160-dimensional vector of discount prices (one for each combination of Checkin Date and Discount Code). The goal is to normalize these prices so that, for each hotel, the lowest valid discount price becomes 0 and the highest becomes 100. Any missing value (indicated by `-1`) should remain unchanged.

**Plan**

1. **Define a Normalization Function:**  
   Create a function (`normalize_row`) that:
   - Filters out the missing values (`-1`) from the row.
   - Computes the minimum and maximum values among the valid discount prices.
   - Applies the normalization formula:
     $$
     \text{normalized\_price} = \frac{(\text{price} - \text{min\_price})}{(\text{max\_price} - \text{min\_price})} \times 100
     $$
   - Handles the case where all valid prices are equal (to avoid division by zero) by setting them to 0.

2. **Apply the Function Row-wise:**  
   Normalize the discount prices for each hotel (i.e., for each row) by applying the function to all columns except the "Hotel Name".

3. **Round and Convert to Integers:**  
   After normalization, round the values to the nearest integer and convert them to an integer type, ensuring that the normalized prices are stored as integers.



In [5]:
def normalize_row(row):
    valid_mask = row != -1
    valid_prices = row[valid_mask]
    
    if valid_prices.empty:
        return row
    
    min_price = valid_prices.min()
    max_price = valid_prices.max()
    
   # Avoid division by zero if all valid prices are identical
    if min_price == max_price:
        row[valid_mask] = 0
    else:
        # Compute the normalized values, round them, and cast to int
        normalized_values = ((row[valid_mask] - min_price) / (max_price - min_price)) * 100
        row[valid_mask] = normalized_values.round(0).astype(int)
    
    return row


pivot_df.iloc[:, 1:] = pivot_df.iloc[:, 1:].apply(normalize_row, axis=1)

for col in pivot_df.columns[1:]:
    pivot_df[col] = pd.to_numeric(pivot_df[col], errors='coerce')
    pivot_df[col] = pivot_df[col].astype("Int64")


display(pivot_df)

,Hotel Name,2015-11-11 - 1,2015-11-11 - 2,2015-11-11 - 3,2015-11-11 - 4,2015-10-14 - 1,2015-10-14 - 2,2015-10-14 - 3,2015-10-14 - 4,2015-11-04 - 1,...,2015-11-03 - 3,2015-11-03 - 4,2015-08-13 - 1,2015-08-13 - 2,2015-08-13 - 3,2015-08-13 - 4,2015-11-25 - 1,2015-11-25 - 2,2015-11-25 - 3,2015-11-25 - 4
0,Aloft Harlem,-1,-1,-1,-1,72,-1,32,-1,60,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,Andaz 5th Avenue - a concept by Hyatt,-1,40,65,-1,-1,68,-1,-1,44,...,-1,-1,-1,-1,-1,0,-1,10,48,48
2,Andaz Wall Street - a concept by Hyatt,6,19,26,-1,37,55,53,-1,-1,...,63,70,-1,-1,-1,-1,2,3,1,-1
3,Baccarat Hotel and Residences New York,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,Bentley Hotel,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,55,55,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,Wyndham Garden Brooklyn Sunset Park,-1,13,19,-1,-1,54,62,75,-1,...,10,-1,69,69,-1,-1,-1,30,33,-1
145,Wyndham Garden Chinatown,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
146,Wyndham Garden Long Island City Manhattan View,31,33,-1,-1,-1,-1,78,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,36,-1,-1
147,Wyndham New Yorker,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


### 3.10 Save to CSV

In [6]:
hotels_clustering_data = "../data/hotels_clustering_data.csv"
pivot_df.to_csv(hotels_clustering_data, index=False)

### 3.11 Hierarchical Clustering

**Task**

Using the normalized discount prices for each hotel, we will perform hierarchical clustering to group hotels that exhibit similar pricing behaviors. We have a 160-dimensional feature vector for each hotel (each dimension corresponds to a specific (Checkin Date, Discount Code) pair).

**Plan**

1. **Prepare the Data:**  
   - Load the saved CSV file (`hotels_clustering_data.csv`).
   - Separate the "Hotel Name" column (for labeling) from the numeric feature columns.

2. **Perform Hierarchical Clustering:**  
   - Use SciPy's `linkage` function with Ward's method (which works well with Euclidean distance) to compute the clustering.
   - Generate a linkage matrix that represents the hierarchical clustering.

3. **Plot the Dendrogram:**  
   - Use SciPy's `dendrogram` function to visualize the hierarchical clustering.
   - Label each leaf in the dendrogram with the corresponding hotel name to help interpret the clusters.


In [7]:
%pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import plotly.figure_factory as ff
from scipy.cluster.hierarchy import linkage
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def create_dendrogram_from_csv(csv_path, color_threshold=825, width=600, height=800):
    """Creates a dendrogram figure."""
    clu_df = pd.read_csv(csv_path)
    hotel_names = clu_df["Hotel Name"].values
    X = clu_df.drop("Hotel Name", axis=1).values
    Z = linkage(X, method='ward')

    fig = ff.create_dendrogram(
        X,
        orientation='left',
        labels=hotel_names,
        color_threshold=color_threshold,
        linkagefun=lambda x: Z
    )

    fig.update_layout(
        width=width,
        height=height,
        margin=dict(l=20, r=20, t=20, b=20) 
    )

    return fig

def display_dendrograms_in_grid(csv_path, color_threshold_list=[825, 750, 625, 500]):
    """Displays dendrograms in a 2x2 grid."""

    fig = make_subplots(rows=2, cols=2, subplot_titles=[f"Cut at ~{ct}" for ct in color_threshold_list])

    for i, color_threshold in enumerate(color_threshold_list):
        row = i // 2 + 1
        col = i % 2 + 1
        dendrogram_fig = create_dendrogram_from_csv(csv_path, color_threshold=color_threshold)

        for trace in dendrogram_fig.data:
            fig.add_trace(trace, row=row, col=col)

        fig.update_xaxes(showticklabels=False, showline=False, zeroline=False, ticks="", row=row, col=col)
        fig.update_yaxes(showticklabels=False, showline=False, zeroline=False, ticks="", row=row, col=col)  
        fig.update_layout(showlegend=False)

    fig.update_layout(
        width=2000, 
        height=1000,
        title_text="Hotel Clustering Dendrograms",
        margin=dict(l=20, r=20, t=60, b=20)
    )

    fig.show()

display_dendrograms_in_grid("../data/hotels_clustering_data.csv")

##### Results analysis

**Hotel Pricing Strategies Over Time: Hierarchical Clustering Analysis**

We performed hierarchical clustering on a dataset of hotels, where each hotel is represented by a **160-dimensional vector** of normalized discount prices. In simpler terms, each hotel’s vector shows *how* it discounts (and by how much) across different dates and discount codes. The dendrogram below clusters these hotels based on their similarity in discounting patterns.

Below, we examine **five different “cut” distances**—825, 750, 625, 500, and an additional view with fewer, broader clusters—and describe the cluster/subgroup formations you see in each figure.


**Overall Explanation of the Dendrogram**
- **X-axis**: The distance (or dissimilarity) at which clusters merge. Larger values mean more dissimilar groups.  
- **Y-axis**: The list of hotels, labeled along the left side.  
- **Colored Branches**: Each color indicates a cluster or subgroup under the specified distance threshold.

In general:  
- Hotels that **merge at smaller distances** (farther to the left in the dendrogram) are quite similar in how they price their discounts.  
- If you follow the dendrogram to the right until a major branch merges, that indicates hotels (or clusters of hotels) that are more dissimilar in their pricing behavior.

---

<div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 5%;">
  <div >
    <h4>Cut at ~825</h4>
    <br>
    <img src="../images/clustering-825.png" alt="Cut at ~825" style="width: 100%; height: auto;">
  </div>
  <div >
   <h4>Cut at ~750</h4>
    <br>
    <img src="../images/clustering-750.png" alt="Cut at ~750" style="width: 100%; height: auto;">
  </div>
  <div >
    <h4>Cut at ~625</h4>
    <br>
    <img src="../images/clustering-625.png" alt="Cut at ~625" style="width: 100%; height: auto;">
  </div>
  <div >
    <h4>Cut at ~500</h4>
    <br>
    <img src="../images/clustering-500.png" alt="Cut at ~500" style="width: 100%; height: auto;">
  </div>
</div>

<br>
<br>
<br>


**Overall Results Explanation:**

The analysis reveals that there are clearly three different popular pricing strategies on the high level. The closer you take a look, each strategy has its own "sub-strategies". These subgroups represent hotels with closely aligned discounting patterns, providing valuable insights for competitive analysis and strategic pricing decisions.

**Possible Meanings of Subgroups:**

* **Competitive Landscape:** Hotels within the same subgroup likely compete directly, as they share similar discount structures and timelines.
* **Revenue Management Strategy:** Subgroups can reflect brand or chain policies, centralized pricing software, or shared revenue management practices.
* **Marketing & Differentiation:** Understanding subgroup membership can inform competitive analysis and strategic pricing decisions, motivating hotels to differentiate or align their pricing.
* **Shared Ownership:** Hotels with shared ownership often have similar pricing.
* **Same Chain Hotels:** Hotel chains often have similar pricing strategies.
* **Close Competitors:** Hotels near each other, that compete for the same customers, will often have similar pricing.


#### diving even deeper - (checking the hotel stars and average price with the pricing strategy)

The results from the last section were interesting, so i decided to run the same clustering but with the hotels stars, prices and discounts to see if some patterns emerge.

The new label now contains:
- stars
- avg price
- avg discount
- avg discount rate 

in this format:

`(stars) - price - discount - discount rate`

example:

(5) - 3898 - 3663 - 6% 

In [13]:
import pandas as pd

pivot_df = pd.read_csv("../data/hotels_clustering_data.csv")
df = pd.read_csv("../data/hotels_data_changed.csv")

pivot_df["Hotel Name"] = pivot_df["Hotel Name"].astype(str).str.strip()
hotel_counts = df["Hotel Name"].value_counts()
top_150_hotels = hotel_counts.head(150).index

summary_df = (
    df[df["Hotel Name"].isin(top_150_hotels)]
    .groupby("Hotel Name")
    .agg({"Original Price": "mean", "Discount Price": "mean", "Hotel Stars": "first"})
    .reset_index()
)

summary_df["Hotel Name"] = summary_df["Hotel Name"].astype(str).str.strip()
summary_df["Original Price"] = summary_df["Original Price"].round(0).astype(int)
summary_df["Discount Price"] = summary_df["Discount Price"].round(0).astype(int)
merged_df = pivot_df.merge(summary_df, on="Hotel Name", how="left")

merged_df["DiscountPerc"] = (((merged_df["Original Price"] - merged_df["Discount Price"]) / merged_df["Original Price"]) * 100).round(0).astype(int)
merged_df["Label"] = merged_df.apply(lambda row: f"({row['Hotel Stars']}) - {row['Original Price']} - {row['Discount Price']} - {row['DiscountPerc']}%", axis=1)

merged_df.head()
merged_df.to_csv("../data/hotels_clustering_data_with_summary.csv", index=False)


In [14]:
import pandas as pd
import plotly.figure_factory as ff
from scipy.cluster.hierarchy import linkage

def create_dendrogram_from_csv(csv_path, color_threshold=825, width=1600, height=1800):
    clu_df = pd.read_csv(csv_path)
    if "Label" in clu_df.columns:
        labels = clu_df["Label"].values
        non_clustering = ["Hotel Name", "Label", "Hotel Stars", "Original Price", "Discount Price", "DiscountPerc"]
    else:
        labels = clu_df["Hotel Name"].values
        non_clustering = ["Hotel Name", "Hotel Stars", "Original Price", "Discount Price", "DiscountPerc"]
    X = clu_df.drop(columns=non_clustering, errors='ignore').values
    Z = linkage(X, method='ward')
    fig = ff.create_dendrogram(
        X,
        orientation='left',
        labels=labels,
        color_threshold=color_threshold,
        linkagefun=lambda x: Z
    )
    fig.update_layout(width=width, height=height)
    fig.show()
    return fig

color_threshold_list = [625]
for color_threshold in color_threshold_list:
    print(f'color_threshold={color_threshold}')
    create_dendrogram_from_csv("../data/hotels_clustering_data_with_summary.csv", color_threshold=color_threshold)


color_threshold=625


###### Hierarchical Clustering with Star Rating, Avg Price, and Discounts

We re-ran clustering with four features:
1. **Hotel star rating**  
2. **Avg nightly price**  
3. **Avg absolute discount**  
4. **Avg discount rate (%)**

Each dendrogram label is `(stars) – price – discount – discount rate`.

###### Takeaways
- data was pretty inconclusive, did not add much.
- **Star Rating is Primary** - hotels with the smae stras are likely to be gouped together but they are all over the place. 
- Hotels in tight clusters likely share near-identical pricing/discount policies, suggesting **direct competition**.  